In [1]:
import numpy as np
from scipy import interpolate
from matplotlib import pyplot as plt

from demo_util import gabor_util
ContrastLandscape = gabor_util.ContrastLandscape

In [2]:
def plot_images(path_image, bg_path_image, path_contrast=None, bg_contrast=None):
    plot_contrast = path_contrast is not None
    
    width = 4.8*5
    num_rows = 4
    if not plot_contrast:
        width = width/2
        num_rows = num_rows//2
    
    plt.figure(figsize=[6.4*2.3, width])
    
    ax = plt.subplot(num_rows, 1, 1)
    ax.imshow(bg_path_image, cmap="gray", origin="upper",
              vmin=-1., vmax=1.)
    
    ax= plt.subplot(num_rows, 1, 2)
    ax.imshow(path_image, cmap="gray", origin="upper",
              vmin=-1., vmax=1.)
    
    if plot_contrast:
        ax = plt.subplot(num_rows, 1, 3)
        ax.imshow(path_contrast, cmap="gray", origin="upper",
                  vmin=0., vmax=1.)
        ax.set_title("Path contrast")

        ax = plt.subplot(num_rows, 1, 4)
        ax.imshow(bg_contrast, cmap="gray", origin="upper",
                  vmin=0., vmax=1.)
        ax.set_title("Background contrast")

    plt.tight_layout()
    plt.show()

In [3]:
from ipywidgets import interactive
import ipywidgets as widgets
import functools

seed = widgets.BoundedIntText(
    value=283214663,
    min=0,
    max=2e32,
    step=1,
)
scale = widgets.IntSlider(
    value=4,
    min=1,
    max=32,
    step=1,
    continuous_update=False,
)
wavelength = widgets.IntSlider(
    value=8,
    min=1,
    max=32,
    step=1,
    continuous_update=False,
)
cell_size = widgets.IntSlider(
    value=32,
    min=8,
    max=64,
    step=1,
    continuous_update=False,
)
num_points = widgets.IntSlider(
    value=12,
    min=3,
    max=32,
    step=1,
    continuous_update=False,
)
path_angle = widgets.FloatSlider(
    value=45,
    min=0,
    max=180,
    step=1.,
    continuous_update=False,
)
angle_noise = widgets.FloatSlider(
    value=10.,
    min=0,
    max=45,
    step=1.0,
    continuous_update=False,
)
path_contrast = widgets.ToggleButton(
    value=False,
    description='Path',
    tooltip="Make the path's contrast random",
)
bg_contrast = widgets.ToggleButton(
    value=False,
    description='Background',
    tooltip="Make the background's contrast random",
)
shared_random = widgets.ToggleButton(
    value=False,
    description='Shared Random',
    tooltip=("Make the path's random contrast equal to the "
             "background's random contrast."),
)
min_contrast = widgets.FloatSlider(
    value=0.,
    min=0.,
    max=1.,
    step=0.02,
    continuous_update=False,
)
max_contrast = widgets.FloatSlider(
    value=1.,
    min=0.,
    max=1,
    step=0.02,
    continuous_update=False,
)
contrast_resolution = widgets.IntSlider(
    value=20,
    min=3,
    max=30,
    step=1,
    continuous_update=False,
)
left_col_phase = widgets.ToggleButton(
    value=True,
    description='Random phase',
    tooltip=("Use random phases"),
)
left_col_align = widgets.ToggleButton(
    value=False,
    description='Align phase',
    tooltip=("Align the gabor widow to match the phase"),
)
right_col_phase = widgets.ToggleButton(
    value=True,
    description='Random phase',
    tooltip=("Use random phases"),
)
right_col_align = widgets.ToggleButton(
    value=True,
    description='Align phase',
    tooltip=("Align the gabor widow to match the phase"),
)

def update_max_range(*args):
    max_contrast.min = min_contrast.value
min_contrast.observe(update_max_range, 'value')

grid_size = 16
kernel_size = 65
start_distance = 128

def plotting_function(path_contrast, bg_contrast, shared_random, contrast_resolution,
                      path_angle, angle_noise, **kwargs):
    contrast_grid_size = (contrast_resolution, contrast_resolution)
    contrast_landscape = ContrastLandscape.FIXED
    if path_contrast:
        contrast_landscape = contrast_landscape | ContrastLandscape.RANDOM_PATH
    if bg_contrast:
        contrast_landscape = contrast_landscape | ContrastLandscape.RANDOM_BACKGROUND
    if shared_random:
        contrast_landscape = contrast_landscape | ContrastLandscape.SHARED_RANDOM
    
    path_image, bg_path_image, path_contrast, bg_contrast = gabor_util.generate_images(
        grid_size=grid_size,
        kernel_size=kernel_size,
        start_distance=start_distance,
        contrast_grid_size=contrast_grid_size,
        contrast_landscape=contrast_landscape,
        generate_contrast_image=True,
        path_angle=np.deg2rad(path_angle),
        angle_noise=np.deg2rad(angle_noise),
        **kwargs,
    )
    plot_images(path_image, bg_path_image, path_contrast, bg_contrast)
    
widget_dict = {
        "seed": seed,
        "scale": scale,
        "wavelength": wavelength,
        "cell_size": cell_size,
        "num_points": num_points,
        "path_angle": path_angle,
        "angle_noise": angle_noise,
        "path_contrast": path_contrast,
        "bg_contrast": bg_contrast,
        "shared_random": shared_random,
        "min_contrast": min_contrast,
        "max_contrast": max_contrast,
        "contrast_resolution": contrast_resolution,
    }

left_widget_dict = widget_dict.copy()
left_widget_dict.update({
    "random_phase": left_col_phase,
    "align_phase": left_col_align,
})

left_buttons = widgets.HBox([left_col_phase, left_col_align])
left_buttons.layout.justify_content = "center"

left_out = widgets.VBox([
    left_buttons,
    widgets.interactive_output(plotting_function, left_widget_dict),
])

right_widget_dict = widget_dict.copy()
right_widget_dict.update({
    "random_phase": right_col_phase,
    "align_phase": right_col_align,
})

right_buttons = widgets.HBox([right_col_phase, right_col_align])
right_buttons.layout.justify_content = "center"

right_out = widgets.VBox([
    right_buttons,
    widgets.interactive_output(plotting_function, left_widget_dict),
])

out = widgets.HBox([left_out, right_out])
out.layout.justify_content = "space-around"

gabor_widget = widgets.VBox([
    widgets.HBox([widgets.Label('Gabor scale (px):'), scale]),
    widgets.HBox([widgets.Label('Gabor wavelength (px):'), wavelength]),
])
path_widget = widgets.VBox([
    widgets.HBox([widgets.Label('Path length (# steps):'), num_points]),
    widgets.HBox([widgets.Label('Path angles (deg):'), path_angle]),
    widgets.HBox([widgets.Label('Path angle noise (deg):'), angle_noise]),
])
contrast_widget = widgets.VBox([
    widgets.HBox([widgets.Label('Random Contrast:'),
                  path_contrast,
                  bg_contrast,
                  shared_random]),
    widgets.HBox([widgets.Label('Max contrast:'), max_contrast]),
    widgets.HBox([widgets.Label('Min contrast:'), min_contrast]),
    widgets.HBox([widgets.Label('Contrast grid'), contrast_resolution]),
])


tabs = widgets.Tab()
tab_titles = ["Gabor", "Path", "Contrast"]
tabs.children = [gabor_widget, path_widget, contrast_widget]
for i, title in enumerate(tab_titles):
    tabs.set_title(i, title)

tabs.layout.margin = "1em"
    
widget_rows = widgets.VBox([
    widgets.HBox([widgets.Label('Seed:'), seed, widgets.Label('Cell size (px):'), cell_size]),
    tabs,
])
all_outputs = widgets.VBox([widget_rows, out])
all_outputs.layout.align_items = "center"

display(all_outputs)